## 1. 코에 동그라미 그리기(1)
-- base 에서 작업합니다. 현대 gpu모델에서 모듈 부딪힘.

In [3]:
%pip install opencv-python
%pip install mediapipe

  Using cached opencv_python-4.8.0.74-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\Users\\user\\miniconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import mediapipe as mp



# 얼굴 찾고, 특징 표시
mp_face_detection = mp.solutions.face_detection #얼굴 검출
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징 표시

# For webcam input:
cap = cv2.VideoCapture(0)
# model_selection=0 -> 2m 내, model_selection=1 -> 5m 내
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
      while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection) # detection 한 모든 부분 표시
               
                #코 좌표 잡기
                nose_coor = detection.location_data.relative_keypoints[2]

                #코의 상대좌표를 이미지 내 좌표로 변환
                h,w,_ = image.shape
                nose_coor = (int(nose_coor.x * w) , int(nose_coor.y * h))

                #코에 동그라미 그리기
                cv2.circle(image,nose_coor,20,(255,0,0),2,cv2.LINE_AA)

                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

## 2. 코에 동그라미 그리기(2) 
- 얼굴 너비에 비례한 코 동그라미 적용

In [5]:
import cv2
import mediapipe as mp



# 얼굴 찾고, 특징 표시
mp_face_detection = mp.solutions.face_detection #얼굴 검출
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징 표시

# For webcam input:
cap = cv2.VideoCapture(0)
# model_selection=0 -> 2m 내, model_selection=1 -> 5m 내
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
      while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection) # detection 한 모든 부분 표시
               
                #코 좌표 잡기
                nose_coor = detection.location_data.relative_keypoints[2]
                ratio = int(80*detection.location_data.relative_bounding_box.width) #얼굴너비를 기준으로 ratio 계산
                #print(detection.location_data.relative_bounding_box.width,ratio)

                #코의 상대좌표를 이미지 내 좌표로 변환
                h,w,_ = image.shape
                nose_coor = (int(nose_coor.x * w) , int(nose_coor.y * h))

                #코에 동그라미 그리기
                cv2.circle(image,nose_coor,ratio,(255,0,0),2,cv2.LINE_AA)

                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

## 3. 코에 돼지코 합성

In [6]:
import cv2
import mediapipe as mp
import numpy as np

#이미지 합성 (원본이미지, 원본x좌표, 원본y 좌표, 합성할 png(4채널)이미지)
def overlay(image, x, y,over_image):
    tmp=over_image
    over_h, over_w, _ = np.shape(tmp)

    w=int(over_w/2)
    h=int(over_h/2)
    mask_img=tmp[:,:,3]/255
    try:
        for c in range(3):
            image[y-h:y+h,x-w:x+w,c]=((tmp[:,:,c]*mask_img)+(image[y-h:y+h,x-w:x+w,c]*(1-mask_img)))#.astype(np.uint8)
    except:
        pass

nose_img=cv2.imread('./nose150.png',cv2.IMREAD_UNCHANGED) # 150x500
left_img=cv2.imread('./left_ear80.png',cv2.IMREAD_UNCHANGED) # 300x100
right_img=cv2.imread('./right_ear80.png',cv2.IMREAD_UNCHANGED) # 300x100

# 얼굴 찾고, 특징 표시
mp_face_detection = mp.solutions.face_detection #얼굴 검출
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징 표시

# For webcam input:
cap = cv2.VideoCapture(0)
# model_selection=0 -> 2m 내, model_selection=1 -> 5m 내
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
      while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                #mp_drawing.draw_detection(image, detection) # detection 한 모든 부분 표시
               
                #코,오른쩍 귀, 왼쪽 귀 좌표 잡기 
                nose_coor = detection.location_data.relative_keypoints[2] # 코
                right_ear = detection.location_data.relative_keypoints[4] # 오른쪽 귀
                left_ear = detection.location_data.relative_keypoints[5]  #왼쪽 귀

                #코,오른쩍 귀, 왼쪽 귀의 이미지 내 실제 좌표(픽셀)
                h,w,_ = image.shape
                nose_coor = (int(nose_coor.x * w) , int(nose_coor.y * h))
                right_ear = (int(right_ear.x * w) , int(right_ear.y * h-80))
                left_ear = (int(left_ear.x * w) , int(left_ear.y * h-80))
             
                #코에 이미지 합성 () overlay(원본이미지, x,y, 합성할 png이미지)  => x,y :합성할 이미지 픽셀 위치
                overlay(image, nose_coor[0], nose_coor[1], nose_img)
                overlay(image, right_ear[0], right_ear[1], right_img)
                overlay(image, left_ear[0], left_ear[1], left_img)

                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

ValueError: not enough values to unpack (expected 3, got 0)

# 